In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import math
import os
import sys
import openpyxl

# --- Configuration ---
workbook_path = 'Ahıskalı, Ayşe Selen-121203103.xlsx' # <--- UPDATED Filename
target_sheet = 'Question'                          # Assume sheet name is the same

# --- File Check ---
if not os.path.exists(workbook_path):
    print(f"FATAL ERROR: Cannot find the Excel file '{workbook_path}'.")
    sys.exit(1)

# --- Initialize variables ---
distance_costs = None
starting_route = None
current_temperature = None
cooling_rate = None
max_iterations = 10
random_sequence = []
starting_cost = None
min_cost_found = float('inf')
best_cost_after_iter1 = None # For U4

# --- Read Data ---
try:
    excel_data = pd.read_excel(workbook_path, sheet_name=target_sheet, header=None, engine='openpyxl')
    dist_matrix_slice = excel_data.iloc[4:10, 1:7] # B5:G10
    numeric_dist_matrix = dist_matrix_slice.applymap(pd.to_numeric, errors='coerce')
    temp_matrix = numeric_dist_matrix.values
    if temp_matrix.shape != (6, 6): print(f"WARNING: Distance matrix shape is {temp_matrix.shape}, expected (6, 6). Check range B5:G10.")
    rows, cols = temp_matrix.shape
    for r in range(rows):
        for c in range(r + 1, cols):
            val_rc = temp_matrix[r, c]; val_cr = temp_matrix[c, r]
            if pd.isna(val_rc) and not pd.isna(val_cr): temp_matrix[r, c] = val_cr
            elif not pd.isna(val_rc) and pd.isna(val_cr): temp_matrix[c, r] = val_rc
    distance_costs = pd.DataFrame(temp_matrix).fillna(0).values
    start_route_series = excel_data.iloc[2, 13:20] # N3:T3
    start_route_nullable = start_route_series.astype('Int64')
    if start_route_nullable.isnull().any(): raise ValueError(f"Initial solution cells (N3:T3) contain invalid data: {start_route_series[start_route_nullable.isnull()].tolist()}")
    starting_route = start_route_nullable.tolist()
    # --- *** CORRECTED Start/End Node for Ayşe Selen *** ---
    start_node, end_node = 3, 3
    if not (starting_route and len(starting_route) == 7 and starting_route[0] == start_node and starting_route[-1] == end_node):
         raise ValueError(f"Initial route {starting_route} (N3:T3) must have 7 nodes and start/end with Node {start_node}.")
    current_temperature = float(excel_data.iloc[11, 1]); cooling_rate = float(excel_data.iloc[15, 1]); # B12, B16
    random_num_series = excel_data.iloc[4:24, 9]; numeric_random_seq = pd.to_numeric(random_num_series, errors='coerce') # J5:J24
    if numeric_random_seq.isnull().any(): raise ValueError(f"Random numbers column (J5:J24) contains invalid data: {random_num_series[numeric_random_seq.isnull()].tolist()}")
    random_sequence = numeric_random_seq.astype(float).tolist()
    if len(random_sequence) < max_iterations * 2: raise ValueError(f"Need {max_iterations * 2} random numbers (J5:J24), only found {len(random_sequence)}.")
except Exception as e: print(f"FATAL ERROR during Excel data processing: {e}"); sys.exit(1)

# --- Helper Function ---
def calculate_tour_cost(tour_nodes, cost_matrix):
    total_distance = 0.0
    try:
        for i in range(len(tour_nodes) - 1):
            from_idx = tour_nodes[i] - 1; to_idx = tour_nodes[i+1] - 1
            if not (0 <= from_idx < cost_matrix.shape[0] and 0 <= to_idx < cost_matrix.shape[1]): raise IndexError(f"Node ID out of bounds ({tour_nodes[i]} or {tour_nodes[i+1]})")
            segment_cost = cost_matrix[from_idx, to_idx]
            if pd.isna(segment_cost) or (segment_cost == 0 and from_idx != to_idx): segment_cost = 0.0
            total_distance += segment_cost
    except Exception as error: print(f"ERROR calculating cost for tour {tour_nodes}: {error}"); return float('inf')
    return total_distance

# --- Cell Mapping ---
output_row_map = { 't': 7, 'rand_idx_swap': 8, 'rand_num_swap': 9, 'swap_node_idx': 10,'new_route': 12, 'new_route_ofv': 12, 'cost_delta': 14, 'accept_direct': 15, 'prob_accept': 16,'rand_idx_accept': 17, 'rand_num_accept': 18, 'accept_via_prob': 19,'current_route': 21, 'current_route_ofv': 21, 'best_route': 22, 'best_route_ofv': 22 }
iter_start_col_idx = 13; iter_column_width = 11; prob_block_offset = 4; ofv_column_offset = 7
initial_cost_target_cell = (2, 20); iter1_best_ofv_target_cell = (3, 20) # U3, U4

# --- Initialize SA ---
active_route = starting_route[:]
starting_cost = calculate_tour_cost(active_route, distance_costs)
if starting_cost == float('inf'): print("FATAL ERROR: Could not calculate cost for the initial route."); sys.exit(1)
active_route_cost = starting_cost; best_route_found = active_route[:]; min_cost_found = active_route_cost
temperature = current_temperature; r_num_idx = 0

# --- Main Loop ---
iteration_log = []
for iter_num in range(max_iterations):
    current_iteration = iter_num + 1; step_results = {}
    iter_base_col = iter_start_col_idx + iter_num * iter_column_width; iter_prob_col = iter_base_col + prob_block_offset; iter_ofv_col = iter_base_col + ofv_column_offset
    if r_num_idx >= len(random_sequence): print(f"STOP: Ran out of random numbers before Iteration {current_iteration}."); break
    temp_this_iter = temperature; step_results[(output_row_map['t'], iter_base_col)] = round(temp_this_iter, 4)
    swap_rand_list_idx = r_num_idx; swap_r_num = random_sequence[swap_rand_list_idx]; r_num_idx += 1; first_swap_node_idx = math.floor(swap_r_num * 4) + 1
    step_results[(output_row_map['rand_idx_swap'], iter_base_col)] = swap_rand_list_idx + 1; step_results[(output_row_map['rand_num_swap'], iter_base_col)] = round(swap_r_num, 4); step_results[(output_row_map['swap_node_idx'], iter_base_col)] = first_swap_node_idx
    candidate_route = active_route[:]; idx_to_swap1 = first_swap_node_idx; idx_to_swap2 = first_swap_node_idx + 1
    if 1 <= idx_to_swap1 <= 4: candidate_route[idx_to_swap1], candidate_route[idx_to_swap2] = candidate_route[idx_to_swap2], candidate_route[idx_to_swap1]
    else: continue
    candidate_route_cost = calculate_tour_cost(candidate_route, distance_costs)
    if candidate_route_cost == float('inf'): continue
    for j, node in enumerate(candidate_route): step_results[(output_row_map['new_route'], iter_base_col + j)] = node
    step_results[(output_row_map['new_route_ofv'], iter_ofv_col)] = round(candidate_route_cost, 1)
    cost_difference = candidate_route_cost - active_route_cost; step_results[(output_row_map['cost_delta'], iter_base_col)] = round(cost_difference, 1)
    move_accepted = False; step_results[(output_row_map['prob_accept'], iter_prob_col)] = ""; step_results[(output_row_map['rand_idx_accept'], iter_prob_col)] = ""; step_results[(output_row_map['rand_num_accept'], iter_prob_col)] = ""; step_results[(output_row_map['accept_via_prob'], iter_prob_col)] = ""
    if cost_difference < 0: move_accepted = True; step_results[(output_row_map['accept_direct'], iter_base_col)] = "YES"
    else:
        step_results[(output_row_map['accept_direct'], iter_base_col)] = "NO"
        if r_num_idx >= len(random_sequence): print(f"STOP: Ran out of random numbers for acceptance in Iteration {current_iteration}."); break
        accept_rand_list_idx = r_num_idx; accept_r_num = random_sequence[accept_rand_list_idx]; r_num_idx += 1; accept_prob = 0.0
        if temp_this_iter > 1e-9:
            try: exponent = -cost_difference / temp_this_iter; accept_prob = 0.0 if exponent < -700 else math.exp(exponent)
            except OverflowError: accept_prob = 0.0
        step_results[(output_row_map['prob_accept'], iter_prob_col)] = round(accept_prob, 4); step_results[(output_row_map['rand_idx_accept'], iter_prob_col)] = accept_rand_list_idx + 1; step_results[(output_row_map['rand_num_accept'], iter_prob_col)] = round(accept_r_num, 4)
        if accept_r_num < accept_prob: move_accepted = True; step_results[(output_row_map['accept_via_prob'], iter_prob_col)] = "YES"
        else: move_accepted = False; step_results[(output_row_map['accept_via_prob'], iter_prob_col)] = "NO"
    if move_accepted:
        active_route = candidate_route[:]; active_route_cost = candidate_route_cost
        if active_route_cost < min_cost_found: min_cost_found = active_route_cost; best_route_found = active_route[:]
    for j, node in enumerate(active_route): step_results[(output_row_map['current_route'], iter_base_col + j)] = node
    step_results[(output_row_map['current_route_ofv'], iter_ofv_col)] = round(active_route_cost, 1)
    for j, node in enumerate(best_route_found): step_results[(output_row_map['best_route'], iter_base_col + j)] = node
    step_results[(output_row_map['best_route_ofv'], iter_ofv_col)] = round(min_cost_found, 1)
    if current_iteration == 1: best_cost_after_iter1 = min_cost_found
    temperature = temperature * cooling_rate; iteration_log.append(step_results)

# --- Write Results ---
try:
    workbook = openpyxl.load_workbook(workbook_path);
    if target_sheet in workbook.sheetnames: worksheet = workbook[target_sheet]
    else: worksheet = workbook.active; print(f"WARNING: Sheet '{target_sheet}' not found. Writing to active sheet '{worksheet.title}'.")
    for step_data in iteration_log:
        for (row_idx, col_idx), value_to_write in step_data.items():
            excel_row = row_idx + 1; excel_col = col_idx + 1
            try: worksheet.cell(row=excel_row, column=excel_col, value=value_to_write)
            except AttributeError: continue
            except Exception as cell_err: print(f"WARNING: Failed to write '{value_to_write}' to cell [{excel_row},{excel_col}]. Error: {cell_err}")
    if starting_cost is not None and starting_cost != float('inf'): init_r, init_c = initial_cost_target_cell; worksheet.cell(row=init_r + 1, column=init_c + 1, value=round(starting_cost, 1)) # U3
    else: print("WARNING: Could not write Original Initial OFV to U3.")
    if best_cost_after_iter1 is not None: iter1_best_r, iter1_best_c = iter1_best_ofv_target_cell; worksheet.cell(row=iter1_best_r + 1, column=iter1_best_c + 1, value=round(best_cost_after_iter1, 1)) # U4
    else: print("WARNING: Could not write Best OFV after Iter 1 to U4.")
    workbook.save(workbook_path)
    print(f"\nProcessing complete. Results written to '{workbook_path}', sheet '{target_sheet}'.")
    print("Please CLOSE and REOPEN the Excel file to view updated results.")
except PermissionError: print(f"\nFATAL ERROR: Permission denied writing to '{workbook_path}'. CLOSE Excel. Check permissions.")
except ImportError: print("\nFATAL ERROR: 'openpyxl' library required but not installed. Run: pip install openpyxl")
except Exception as e: print(f"\nFATAL ERROR during Excel writing: {e}");


Processing complete. Results written to 'Ahıskalı, Ayşe Selen-121203103.xlsx', sheet 'Question'.
Please CLOSE and REOPEN the Excel file to view updated results.
